# FIX Vocabulary

FIX Vocabulary is collection of stable semantic business concepts in FIX Protocol. The concepts are independent of encoding.

This workbook loads a local file and demonstrates simple queries. The file uses semantic notations RDF and SKOS.

In [1]:
import sys
from gastrodon import *
from rdflib import *
import pandas as pd
pd.options.display.width=120
pd.options.display.max_colwidth=100

In [2]:
g=Graph()

Load local FIX Vocabulary file in turtle format into a graph provided RDFLib.

In [3]:
g.parse("fix-vocabulary.ttl",format="turtle")

<Graph identifier=N8e1e71f11cfd4ba78fc4f8196c49c5f8 (<class 'rdflib.graph.Graph'>)>

In [4]:
len(g)

1865

In [5]:
g.bind("owl","http://www.w3.org/2002/07/owl#")
g.bind("dct", "http://purl.org/dc/terms/")
g.bind("skos","http://www.w3.org/2004/02/skos/core#")
e=LocalEndpoint(g)

Query the ontology metadata

In [6]:
s=e.select("""
   SELECT ?s {
      ?s rdf:type owl:Ontology .
    }
""")
s

,s
0,https://fixprotocol.io/vocabulary


In [7]:
ontology=one(s)
ontology

rdflib.term.URIRef('https://fixprotocol.io/vocabulary')

In [8]:
meta=e.select("""
    SELECT ?p ?o 
    WHERE {
        ?_ontology ?p ?o .
    } 
""")
meta

,p,o
0,dct:rights,Copyright 2018 FIX Trading Community
1,dct:license,Apache License Version 2.0
2,owl:imports,http://www.w3.org/2004/02/skos/core
3,dct:title,FIX Vocabulary
4,dct:date,2018-06-13 13:30:36.633588+00:00
5,rdf:type,owl:Ontology


### Show all concepts

Query all terms in the ontology

In [9]:
terms=e.select("""
    SELECT ?term ?definition ?from 
    WHERE {
        ?concept  skos:prefLabel ?term . 
        ?concept skos:definition ?definition .
        OPTIONAL { ?concept dct:isPartOf ?from . }
    }
""")
terms

,term,definition,from
0,OTC,"Used to report into an exchange trades or prices that were agreed directly, outside of the exch...",None
1,Even swap,An FX Swap where the given amount to be bought and sold is the same on the near and far legs. ...,None
2,Marked To Market Yield,An adjustment in the valuation of a securities portfolio to reflect the current market values o...,None
3,Yield Difference,The price of a strategy (multileg) order is given as a spread between the two legs.,None
4,Submission to Clearing,The timestamp when the trade was officially acknowledged by the Clearing House,EP77
5,Suspended,The order is not eligible for trading. This usually happens as a result of a verbal or otherwis...,None
6,Automated Floor Order Routing,The use of electronic devices or systems to capture orders and route the resulting trades to do...,None
7,Triggered or Activated by System,An execution instruction that indicates that the sending system activated or updated the order....,None
8,American style option,An option that can be exercised at anytime before its expiration date.,None
9,Request to Intermediary,"Used in a model where an intermediary, e.g. clearing house, is involved in communicating alloca...",None


### Selective Query

Query all terms like "Option"

In [10]:
options=e.select("""
    SELECT ?term ?definition 
    WHERE {
        ?concept  skos:prefLabel ?term . 
        ?concept skos:definition ?definition .
        FILTER regex(?term, "Option") 
    }
""")
options

,term,definition
0,Binary Option,A binary option is a cash settled option that has a discontinuous payoff. Binary options come i...
1,Binary All or None Option,Fixed pay out if the underlying settles on a predefined trigger price.
2,Binary Barrier Option,A digital option which becomes active or inactive based on the crossing of a barrier. Events ar...
3,Look-back Option,The underlying price is set to the optimal value of the daily settlement prices over a specifie...
4,One Touch Option,"One Touch further defines the terms, usually specific dates and times, for when the option will..."
5,Capped Call Option,The option has a linear payout (like a vanilla) up to a capped amount as specified by a cap price.
6,Capped Barrier Option,A capped option which becomes active or inactive based on the crossing of a barrier.
7,Reset Barrier Option,After hitting the barrier the next specified barrier goes into effect. The payoff depends on th...
8,Binary Range Option,Fixed pay out if the underlying settles between an upper and lower trigger price.
9,Vanilla Payout Option,The payout amount is determined by the difference between the strike and the underlying.
